In [ ]:
import numpy as np
import pandas as pd
import cupy as cp
from typing import List, Tuple, Optional, Dict
import os
import matplotlib.pyplot as plt
import japanize_matplotlib
from scipy import signal, integrate
import cv2
from scipy.stats import entropy
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import UnivariateSpline

from skimage.metrics import structural_similarity as ssim
from skimage.color import rgb2lab
from scipy.spatial import ConvexHull

import pyVHR as vhr
from pyVHR.extraction.sig_processing import SignalProcessing
from pyVHR.plot.visualize import *
from pyVHR.BVP import *
vhr.plot.VisualizeParams.renderer = 'notebook'

import mediapipe as mp

In [ ]:
# 入力とする動画と動画のファイル名を取得
root_dir = "experimentData\\"
data_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
movie_paths = []
movie_names = []
true_value_csv_array = []
true_value_rri_csv_array = []
print("動画ディレクトリ:", data_dirs)

for i in range(len(data_dirs)):
    data_dir = data_dirs[i]

    # 動画ファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.avi')]
    movie_paths.extend(movie_files)

    movie_name = os.path.basename(data_dir)
    movie_names.append(movie_name)

    # ppgファイルのパスを取得
    movie_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.csv')]
    true_value_csv_array = [f.replace('.avi', '.csv') for f in movie_paths if f.endswith('.avi')]
    true_value_rri_csv_array.append(os.path.join(data_dir, 'RRI_Simple_' + movie_name + '.csv'))


f_1_ffi = 0.0399  # LFのはじめ
f_2 = 0.151  # LFの終わり、HFのはじめ
f_3 = 0.401  # HFの終わり

start_index = 0
# start_index = 17
end_index = 2
# end_index = len(movie_paths)
# start_index = end_index - 1  # 最後の動画のみを対象とする

data_dirs = data_dirs[start_index:end_index]
movie_paths = movie_paths[start_index:end_index]
movie_names = movie_names[start_index:end_index]
true_value_csv_array = true_value_csv_array[start_index:end_index]
true_value_rri_csv_array = true_value_rri_csv_array[start_index:end_index]

print(f"data_dirs: {data_dirs}")
print(f"movie_paths: {movie_paths}")
print(f"movie_names: {movie_names}")
print(f"true_value_csv_array: {true_value_csv_array}")
print(f"true_value_rri_csv_array: {true_value_rri_csv_array}")

In [ ]:
SAVE_DIR  = "POSAccuracyEvalu"
SPLIT_TIME = 90
WINDOW_SIZES = [2, 4, 6, 8, 10]
STRIDES = [2]
RPPG_ACCURACY_EVAL_DIR = "rppgAccuracyEvalu"

成果物
- 1フレームずつのRGBシグナル

## windowごとにRGB信号を分類

In [ ]:
def array_to_full_string(arr):
    """NumPy配列を省略なしの文字列に変換"""
    if isinstance(arr, str):
        return arr
    elif isinstance(arr, (np.ndarray, list)):
        # NumPy配列またはリストを完全な文字列に変換
        arr_np = np.array(arr)
        return np.array2string(arr_np, threshold=np.inf, max_line_width=np.inf, separator=' ')
    else:
        return str(arr)

In [ ]:
class StrideSegmentCalculator:
    def __init__(
        self,
        window_sizes: List[float] = [2, 3, 4, 5],
        strides: List[float] = [0.1, 0.5, 1, 1.5, 2],
    ):
        """
        Parameters:
        -----------
        window_sizes : List[float]
            窓幅（秒）のリスト
        strides : List[float]
            移動秒数のリスト
        """
        self.window_sizes = window_sizes
        self.strides = strides

    def calculate_overlap(self, window_size: float, stride: float) -> float:
        """
        窓幅と移動秒数からオーバーラップ率を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）

        Returns:
        --------
        float
            オーバーラップ率（%）
        """
        if stride >= window_size:
            return 0
        overlap = (window_size - stride) / window_size * 100
        return round(overlap, 2)

    def calculate_segments(
        self, window_size: float, stride: float, total_frames: int, fps: int
    ) -> List[Tuple[int, int]]:
        """
        フレーム数から解析区間を計算

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        stride : float
            移動秒数（秒）
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        List[Tuple[int, int]]
            各区間の(開始フレーム, 終了フレーム)のリスト
        """
        frames_per_window = round(window_size * fps)
        frames_per_stride = round(stride * fps)

        segments = []
        start_frame = 0

        while start_frame + frames_per_window <= total_frames:
            segments.append((start_frame, start_frame + frames_per_window))
            start_frame += frames_per_stride

        return segments

    def create_analysis_dataframe(self, total_frames: int, fps: int) -> pd.DataFrame:
        """
        全ての窓幅と移動秒数の組み合わせに対してDataFrameを生成

        Parameters:
        -----------
        total_frames : int
            総フレーム数
        fps : int
            フレームレート

        Returns:
        --------
        pd.DataFrame
            各条件でのセグメント情報を含むDataFrame
            columns: window_size, stride, overlap, segment_number, frame_start, frame_end
        """
        data_dict = {
            "window_size": [],
            "stride": [],
            "overlap": [],
            "segment_number": [],
            "frame_start": [],
            "frame_end": [],
        }

        for window_size in self.window_sizes:
            for stride in self.strides:
                overlap = self.calculate_overlap(window_size, stride)
                segments = self.calculate_segments(
                    window_size, stride, total_frames, fps
                )

                for i, (start_frame, end_frame) in enumerate(segments):
                    data_dict["window_size"].append(window_size)
                    data_dict["stride"].append(stride)
                    data_dict["overlap"].append(overlap)
                    data_dict["segment_number"].append(i)
                    data_dict["frame_start"].append(start_frame)
                    data_dict["frame_end"].append(end_frame)

        return pd.DataFrame(data_dict)


class PulseAnalysisDataStrides:
    def __init__(self, window_sizes, strides):
        # 窓枠とストライドの値を定義
        self.window_sizes = window_sizes
        self.strides = strides

        # accuracyのみを格納するDataFrameを初期化
        self.results = pd.DataFrame(
            index=pd.Index(self.window_sizes, name="window_size"),
            columns=pd.Index(self.strides, name="strides"),
        )

    def add_accuracy(self, window_size: float, strides: int, accuracy: float):
        """
        精度データを追加する

        Parameters:
        -----------
        window_size : float
            窓幅（秒）
        strides : int
            ストライド（s）
        accuracy : float
            精度値
        """
        self.results.loc[window_size, strides] = accuracy

    def _create_heatmap_dataframe(self):
        """
        ヒートマップ用のDataFrameを作成する内部メソッド

        Returns:
        --------
        pd.DataFrame
            ヒートマップ用に整形されたDataFrame
        """
        data = {"stride": self.strides}
        for window_size in self.window_sizes:
            data[window_size] = [
                self.results.loc[window_size, stride] for stride in self.strides
            ]
        df = pd.DataFrame(data).set_index("stride").T
        return df

    def save_heatmap(
        self,
        title: str,
        save_path: str,
        figsize: tuple = (10, 8),
        cmap: str = "YlGnBu",
        colorbar_label: str = "MAE",
    ):
        """
        ヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        cmap : str, optional
            カラーマップ (default: 'YlGnBu')
        colorbar_label : str, optional
            カラーバーのラベル (default: 'MAE')
        """
        df = self._create_heatmap_dataframe()

        # ヒートマップを作成
        plt.figure(figsize=figsize)
        sns.heatmap(
            df, annot=True, fmt=".4f", cmap=cmap, cbar_kws={"label": colorbar_label}
        )
        plt.title(f"{title}")
        plt.xlabel("Stride [s]")
        plt.ylabel("Window Size [s]")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.close()

    def save_heatmap_std(self, title: str, save_path: str, figsize: tuple = (10, 8)):
        """
        標準偏差のヒートマップを作成して保存する

        Parameters:
        -----------
        title : str
            プロットのタイトル
        save_path : str
            保存先のパス
        figsize : tuple, optional
            図のサイズ (default: (10, 8))
        """
        self.save_heatmap(
            title,
            save_path,
            figsize,
            cmap="Reds",
            colorbar_label="Standard Deviation",
        )

In [ ]:
all_results_list = []
for i in range(len(movie_paths)):
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    window_signals_data_csv_path = os.path.join(rootDir, RPPG_ACCURACY_EVAL_DIR, f'window_signals_{dataName}.csv')
    window_signals_df  = pd.read_csv(window_signals_data_csv_path)
    print(window_signals_df.head())


## 窓ごとにBVP解析
初期か処理のセクション2で同じことやってるので流用

In [ ]:
for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    window_analysis_data_csv_path = os.path.join(rootDir, RPPG_ACCURACY_EVAL_DIR, f'window_signals_{dataName}.csv')
    window_analysis_df = pd.read_csv(window_analysis_data_csv_path)
    print(window_analysis_df.head())

### 窓ごとに輝度変動成分を抽出

In [ ]:
def analyze_window_fft(values, fps):
    """
    時系列データの最大周波数とスペクトル情報を計算する関数

    Parameters:
    -----------
    values : array-like
        分析対象の時系列データ
    fps : int
        サンプリング周波数（1秒あたりのフレーム数）

    Returns:
    --------
    dict
        以下のキーを含む辞書：
        - 'max_freq': 検出された最大周波数
        - 'max_amplitude': 最大周波数のときの振幅
        - 'frequencies': 周波数配列（正の周波数のみ）
        - 'amplitudes': 振幅配列（正の周波数のみ）
        - 'power_spectrum': パワースペクトル
        - 'dominant_freqs': 上位5つの卓越周波数とその振幅
        - 'spectral_centroid': スペクトル重心
        - 'spectral_bandwidth': スペクトル帯域幅
        - 'total_power': 全体のパワー
    """
    # データをnumpy配列に変換
    values = np.array(values)

    # ゼロパディングで分解能を向上（窓長の8倍）
    n_pad = len(values) * 8

    # ハミング窓を適用（オプション：コメントアウトされている）
    # window = np.hamming(len(values))
    # windowed_data = values * window

    # FFTを実行（ゼロパディング適用）
    fft_result = np.fft.fft(values, n=n_pad)
    fft_freq = np.fft.fftfreq(n_pad, 1 / fps)

    # 正の周波数のみを取得
    positive_freq_idx = fft_freq > 0
    positive_fft = np.abs(fft_result[positive_freq_idx])
    positive_freq = fft_freq[positive_freq_idx]
    
    # パワースペクトルを計算
    power_spectrum = positive_fft ** 2

    # 最大周波数の検出と補間
    max_idx = np.argmax(positive_fft)
    max_amplitude = positive_fft[max_idx]
    
    if 0 < max_idx < len(positive_fft) - 1:
        # 3点を使用した放物線補間
        alpha = positive_fft[max_idx - 1]
        beta = positive_fft[max_idx]
        gamma = positive_fft[max_idx + 1]
        peak_pos = 0.5 * (alpha - gamma) / (alpha - 2 * beta + gamma)

        # 補間された周波数と振幅
        freq_resolution = fps / n_pad
        max_freq = positive_freq[max_idx] + peak_pos * freq_resolution
        
        # 補間された振幅（放物線の頂点）
        max_amplitude = beta - 0.25 * (alpha - gamma) * peak_pos
    else:
        max_freq = positive_freq[max_idx]
    
    # 上位5つの卓越周波数を検出
    top_indices = np.argsort(positive_fft)[-5:][::-1]
    dominant_freqs = [(positive_freq[idx], positive_fft[idx]) for idx in top_indices]
    
    # スペクトル特徴量の計算
    # スペクトル重心（周波数の重み付き平均）
    spectral_centroid = np.sum(positive_freq * positive_fft) / np.sum(positive_fft)
    
    # スペクトル帯域幅（重心からの重み付き分散）
    spectral_bandwidth = np.sqrt(
        np.sum(((positive_freq - spectral_centroid) ** 2) * positive_fft) / np.sum(positive_fft)
    )
    
    # 全体のパワー
    total_power = np.sum(power_spectrum)
    
    # 結果を辞書にまとめる
    result = {
        'max_freq': max_freq,
        'max_amplitude': max_amplitude,
        'frequencies': positive_freq,
        'amplitudes': positive_fft,
        'power_spectrum': power_spectrum,
        'dominant_freqs': dominant_freqs,
        'spectral_centroid': spectral_centroid,
        'spectral_bandwidth': spectral_bandwidth,
        'total_power': total_power,
        'freq_resolution': fps / n_pad,  # 周波数分解能
        'nyquist_freq': fps / 2  # ナイキスト周波数
    }
    
    return result

In [ ]:
# メソッドの組み合わせを定義
methodCombinations = [
    ['cuda', cupy_POS, "cupy_POS"]
]

for i in range(len(movie_paths)):
    all_window_results = []
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # window_signalsの読み込み
    os.makedirs(os.path.join(rootDir, SAVE_DIR), exist_ok=True)
    window_analysis_data_path = os.path.join(rootDir, RPPG_ACCURACY_EVAL_DIR, f'{"window_analysis_" + dataName}.csv')
    window_analysis_data_df  = pd.read_csv(window_analysis_data_path)
    print(f'Loaded window signals data from {window_analysis_data_path}, {len(window_analysis_data_df)} rows')

    # bvp_methodが'cupy_POS'のデータのみを抽出
    pos_window_analysis_data_df = window_analysis_data_df[window_analysis_data_df['bvp_method'] == 'cupy_POS'].copy()
    
    window_index  = 0
    for idx, row in pos_window_analysis_data_df.iterrows():
        window_size = row['window_size']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        stride = row['stride']
        original_bpm_mae = row['bpm_MAE']

        window_start_time = frame_start / fps
        window_end_time = frame_end / fps

        print(f"\nウィンドウ {idx}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {int(window_start_time // 60)}m{window_start_time % 60:.2f}s-{int(window_end_time // 60)}m{window_end_time % 60:.2f}s")

        r_signal_in_window = row['r_signal_in_window']
        g_signal_in_window = row['g_signal_in_window']
        b_signal_in_window = row['b_signal_in_window']
        ecg_bpm_in_window = row['ecg_bpm_in_window']

        # 文字列をNumPy配列に変換
        r_signal_in_window = np.fromstring(r_signal_in_window[1:-1], sep=' ')
        g_signal_in_window = np.fromstring(g_signal_in_window[1:-1], sep=' ')
        b_signal_in_window = np.fromstring(b_signal_in_window[1:-1], sep=' ')
        ecg_bpm_in_window = np.fromstring(ecg_bpm_in_window[1:-1], sep=' ')
        ecg_bpm_in_window_mean = np.nanmean(ecg_bpm_in_window)

        if len(r_signal_in_window) == 0 or len(g_signal_in_window) == 0 or len(b_signal_in_window) == 0:
            raise ValueError("RGB信号が窓内に存在しません")
        
        rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
        rgb_cupy = cp.asarray(rgb_signal)
        
         # POS法のパラメータ
        eps = 10**-9
        X = rgb_cupy
        fps_cupy = cp.float32(fps)
        e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
        w = int(1.6 * fps_cupy)  # window length

        # 固定の拍動ベクトル(論文の式30)
        u_pbv = cp.array([0.33, 0.77, 0.53], dtype=cp.float32)

        # 肌色調ベクトル
        # r_signal_in_window_square = r_signal_in_window ** 2
        # g_signal_in_window_square = g_signal_in_window ** 2
        # b_signal_in_window_square = b_signal_in_window ** 2

        # normalized_r_signal_in_window = r_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)
        # normalized_g_signal_in_window = g_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)
        # normalized_b_signal_in_window = b_signal_in_window / np.sqrt(r_signal_in_window_square + g_signal_in_window_square + b_signal_in_window_square)

        # skin_vector_array = np.array([normalized_r_signal_in_window, normalized_g_signal_in_window, normalized_b_signal_in_window])
        # skin_vector = cp.mean(cp.asarray(skin_vector_array), axis=1)

        # POS法の標準化された肌色ベクトル(uskin)と血液量パルスベクトル(upbv)に直交する法線ベクトルを計算

        # u_skin = skin_vector / (cp.linalg.norm(skin_vector) + eps)
        u_skin = [0.37,0.56,0.75]
        # u_pbvとu_skinに直交するベクトルを求める
        uskin = cp.asnumpy(u_skin)
        upbv = cp.asnumpy(u_pbv)
        v_n = cp.cross(u_skin, u_pbv)
        normalize_v_n = v_n / cp.linalg.norm(v_n)

        # 投影行列P(1x3の行ベクトル)
        P = cp.reshape(normalize_v_n, (1, 3))

        normalize_v_n_np = cp.asnumpy(normalize_v_n)
        print(f"肌色調ベクトル uskin: {uskin}, 法線ベクトル v_n: {normalize_v_n_np}")

        # 検証
        dot_u_skin = np.dot(uskin, normalize_v_n_np)
        dot_upbv = np.dot(upbv, normalize_v_n_np)
        print("u_skin・v_n:", dot_u_skin)
        print("u_pbv・v_n:", dot_upbv)
        
        # 投影行列P(1x3の行ベクトル)
        P = cp.reshape(normalize_v_n, (1, 3))

        # 初期化
        intensity_signal = cp.zeros((e, f))

        # スライディングウィンドウループ
        for n in cp.arange(w, f):
            m = n - w + 1
            
            # 時間的正規化
            Cn = X[:, :, m:(n + 1)]
            M = 1.0 / (cp.mean(Cn, axis=2) + eps)
            M_expanded = cp.expand_dims(M, axis=2)
            Cn = cp.multiply(M_expanded, Cn)
            
            # 投影(uskinとupbvに直交する方向に投影してi(t)を抽出)
            # Cn shape: (e, 3, window_length)
            # P shape: (1, 3)
            # 投影結果 shape: (e, window_length)
            for estimator_idx in range(e):
                
                projected = cp.dot(P, Cn[estimator_idx, :, :]).flatten()
                # ゼロ平均化
                projected = projected - cp.mean(projected)
                # オーバーラップ加算
                intensity_signal[estimator_idx, m:(n + 1)] = cp.add(
                    intensity_signal[estimator_idx, m:(n + 1)], 
                    projected
                )
                
        # 最終的な強度変動信号をプロット
        intensity_np = cp.asnumpy(intensity_signal[0, :])

        bvp_cupy = intensity_signal
        bvp_numpy = cp.asnumpy(bvp_cupy)

        raw_bvp_signal = [bvp_numpy]
        bvp_signal = [bvp_numpy.copy()]

        # 後処理フィルタリング
        bvp_signal = vhr.BVP.apply_filter(
            bvp_signal,
            vhr.BVP.BPfilter,
            params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
        )

        bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

        raw_bvp_signal_in_window = raw_bvp_signal[0] if len(bvp_signal) > 0 else None
        filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

        
        # FFT解析
        raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
        filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
        fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

        # MAEの計算
        rppg_bpm = fft_result_dic['max_freq'] * 60
        rppg_freq = fft_result_dic['frequencies']
        rppg_amplitude = fft_result_dic['amplitudes']
        rppg_pwd = fft_result_dic['power_spectrum']

        bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

        print(f"\n結果サマリー:")
        print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
        print(f"  rPPG BPM: {rppg_bpm:.2f}")
        print(f"  MAE: {bpm_MAE:.2f}, 元の窓のMAE: {original_bpm_mae:.2f}, 差分: {bpm_MAE - original_bpm_mae:.2f}")

        # fig, axes = plt.subplots(1, 1, figsize=(15, 4))
        # axes.plot(intensity_np, color='red', label='Intensity Signal i(t) - Full', linewidth=1.5)
        # axes.set_title(f'Window {idx} - Full Intensity Signal (orthogonal to uskin and upbv)\nMAE: {bpm_MAE:.2f}, 元の窓のMAE: {original_bpm_mae:.2f}, 差分: {bpm_MAE - original_bpm_mae:.2f}')
        # axes.set_xlabel('Frame')
        # axes.set_ylabel('Amplitude')
        # axes.legend()
        # axes.grid(True, alpha=0.3)
        # plt.tight_layout()
        # plt.show()

        # 窓情報を保存
        window_info = {
            'window_index': window_index,
            'window_size': window_size,
            'stride': stride,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'window_start_time': window_start_time,
            'window_end_time': window_end_time,
            'r_signal_in_window': array_to_full_string(r_signal_in_window),
            'g_signal_in_window': array_to_full_string(g_signal_in_window),
            'b_signal_in_window': array_to_full_string(b_signal_in_window),
            'raw_intensity_in_window': raw_bvp_signal_in_window,
            'filtered_intensity_in_window': filtered_bvp_signal_in_window,
            'ecg_bpm_in_window': ecg_bpm_in_window,
            'ecg_bpm_mean': ecg_bpm_in_window_mean,
            'rppg_bpm': rppg_bpm,
            'bpm_MAE': bpm_MAE,
            'original_bpm_MAE': original_bpm_mae,
            'max_freq': fft_result_dic['max_freq'],
            'max_amplitude': fft_result_dic['max_amplitude'],
            'spectral_centroid': fft_result_dic['spectral_centroid'],
            'spectral_bandwidth': fft_result_dic['spectral_bandwidth'],
            'total_power': fft_result_dic['total_power']
        }
        all_window_results.append(window_info)
        window_index += 1
    
    # 全結果をDataFrameに変換して保存
    results_df = pd.DataFrame(all_window_results)
    results_save_dir = os.path.join(rootDir, SAVE_DIR)
    os.makedirs(results_save_dir, exist_ok=True)
    results_csv_path = os.path.join(results_save_dir, f'window_intensity_{dataName}.csv')
    # results_df.to_csv(results_csv_path, index=False, encoding='utf-8-sig')
    print(f"\nFFT結果をCSVに保存: {results_csv_path}")  

### POSの射影行列を系統的に変えることで精度が向上するかをテスト

In [ ]:
# 投影行列のバリエーションを定義
PROJECTION_MATRICES = {
    'original': np.array([[0, 1, -1], [-2, 1, 1]]),
    'modified_1': np.array([[-1, 0, 1], [1, -2, 1]]),
    'modified_2': np.array([[1, -1, 0], [1, 1, -2]]),
}

methodCombinations = [
    ['cuda', cupy_POS, "cupy_POS"]
]

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    window_analysis_path = os.path.join(rootDir, RPPG_ACCURACY_EVAL_DIR, f'window_analysis_{dataName}.csv')
    window_analysis_df = pd.read_csv(window_analysis_path)

    # 手法がcupy_POSのデータを抽出
    high_mae_df = window_analysis_df[window_analysis_df['bvp_method'] == 'cupy_POS'].copy()
    print(f'\n高MAEデータ - {dataName}, 件数: {len(high_mae_df)}')

    # 高MAEデータのbpm_MAEの窓長ごとの平均値を計算
    if len(high_mae_df) > 0:
        mean_high_mae = high_mae_df.groupby('window_size')['bpm_MAE'].mean()
        print(f'  高MAEデータの平均MAE:\n{mean_high_mae}')
    else:
        print('  高MAEデータが存在しません')

    # 結果保存用のリスト
    results_list = []
    
    # high_mae_dfの各行を処理
    for idx, row in high_mae_df.iterrows():
        window_index = row['window_index']
        window_size = row['window_size']
        stride = row['stride']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        window_start_time = row['window_start_time']
        window_end_time = row['window_end_time']
        ecg_bpm_in_window_mean = row['ecg_bpm_mean']
        bpm_MAE = row['bpm_MAE']
        if bpm_MAE < 5.0:
            print(f"スキップ: ウィンドウ {window_index} はMAEが低いため ({bpm_MAE:.2f})")
            continue

        print(f"\nウィンドウ {window_index}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # RGB信号を抽出
        r_signal_in_window = np.fromstring(row['r_signal_in_window'][1:-1], sep=' ')
        g_signal_in_window = np.fromstring(row['g_signal_in_window'][1:-1], sep=' ')
        b_signal_in_window = np.fromstring(row['b_signal_in_window'][1:-1], sep=' ')

        print(f'  R信号長: {len(r_signal_in_window)}, G信号長: {len(g_signal_in_window)}, B信号長: {len(b_signal_in_window)}')

        # rgbからBVPを計算
        rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
        print(f"\nRGB信号の形状: {rgb_signal.shape}")

        signal_length = rgb_signal.shape[2]
            
        filtered_signal = [rgb_signal]
        
        for proj_name, P in PROJECTION_MATRICES.items():
            
            # ============================================================================
            # 【重要】POS法を直接実装(調整可能)
            # ============================================================================
            import cupy as cp

            # CuPy配列に変換
            rgb_cupy = cp.asarray(rgb_signal)

            # POS法のパラメータ
            eps = 10**-9
            X = rgb_cupy
            fps_cupy = cp.float32(fps)
            e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
            w = int(1.6 * fps_cupy)  # window length

            # 投影行列P(現在のパターンを使用)
            P_cupy = cp.asarray(P)
            Q = cp.stack([P_cupy for _ in range(e)], axis=0)

            # 初期化
            H = cp.zeros((e, f))

            # 診断情報を保存するリスト
            alpha_list = []
            M_list = []
            S1_list = []
            S2_list = []
            alpha_S2_list = []
            window_indices = []

            # スライディングウィンドウループ
            for n in cp.arange(w, f):
                m = n - w + 1
                
                # 時間的正規化
                Cn = X[:, :, m:(n + 1)]
                M = 1.0 / (cp.mean(Cn, axis=2) + eps)
                M_expanded = cp.expand_dims(M, axis=2)
                Cn = cp.multiply(M_expanded, Cn)
                
                # Mの値を保存
                M_list.append(cp.asnumpy(M))
                
                # 投影
                S = cp.dot(Q, Cn)
                S = S[0, :, :, :]
                S = cp.swapaxes(S, 0, 1)
                
                # チューニング
                S1 = S[:, 0, :]
                S2 = S[:, 1, :]
                alpha = cp.std(S1, axis=1) / (eps + cp.std(S2, axis=1))
                
                # S1とS2を保存
                S1_list.append(cp.asnumpy(S1))
                S2_list.append(cp.asnumpy(S2))
                
                # alphaの値を保存
                alpha_list.append(cp.asnumpy(alpha))
                
                alpha_expanded = cp.expand_dims(alpha, axis=1)
                alpha_S2 = alpha_expanded * S2
                
                # alpha*S2を保存
                alpha_S2_list.append(cp.asnumpy(alpha_S2))
                
                window_indices.append(int(n))
                
                Hn = cp.add(S1, alpha_S2)
                Hnm = Hn - cp.expand_dims(cp.mean(Hn, axis=1), axis=1)
                
                # オーバーラップ加算
                H[:, m:(n + 1)] = cp.add(H[:, m:(n + 1)], Hnm)

            # NumPy配列に戻す
            bvp_cupy = H
            bvp_numpy = cp.asnumpy(bvp_cupy)

            raw_bvp_signal = [bvp_numpy]
            bvp_signal = [bvp_numpy.copy()]

            # 後処理フィルタリング
            bvp_signal = vhr.BVP.apply_filter(
                bvp_signal,
                vhr.BVP.BPfilter,
                params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
            )

            bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

            raw_bvp_signal_in_window = raw_bvp_signal[0] if len(bvp_signal) > 0 else None
            filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

            # FFT解析
            raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
            filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
            fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

            # MAEの計算
            rppg_bpm = fft_result_dic['max_freq'] * 60
            rppg_freq = fft_result_dic['frequencies']
            rppg_amplitude = fft_result_dic['amplitudes']
            rppg_pwd = fft_result_dic['power_spectrum']

            bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

            print(f"\n結果サマリー [{proj_name}]:")
            print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
            print(f"  rPPG BPM: {rppg_bpm:.2f}")
            print(f"  MAE: {bpm_MAE:.2f}")
            
            # 結果を辞書形式でリストに追加
            result_dict = {
                'window_index': window_index,
                'projection': proj_name,
                'window_size': window_size,
                'stride': stride,
                'frame_start': frame_start,
                'frame_end': frame_end,
                'window_start_time': window_start_time,
                'window_end_time': window_end_time,
                'ecg_bpm': ecg_bpm_in_window_mean,
                'rppg_bpm': rppg_bpm,
                'bpm_MAE': bpm_MAE
            }
            results_list.append(result_dict)
    
    # 全ウィンドウの処理が終わった後、DataFrameに変換
    results_df = pd.DataFrame(results_list)
    
    # 結果をCSVに保存
    output_path = os.path.join(rootDir, SAVE_DIR, f'projection_comparison_{dataName}.csv')
    results_df.to_csv(output_path, index=False)
    print(f"\n結果を保存しました: {output_path}")
    
    # サマリーを表示
    print(f"\n{'='*60}")
    print("全投影行列パターンの比較サマリー:")
    print(f"{'='*60}")
    summary = results_df.groupby('projection')['bpm_MAE'].agg(['mean', 'std', 'min', 'max'])
    print(summary)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    projection_comparison_path = os.path.join(rootDir, SAVE_DIR, f'projection_comparison_{dataName}.csv')
    
    # CSVファイルが存在するか確認
    if not os.path.exists(projection_comparison_path):
        print(f'ファイルが見つかりません: {projection_comparison_path}')
        continue
    
    # データを読み込む
    df = pd.read_csv(projection_comparison_path)
    print(f'\n読み込んだデータ件数: {len(df)}')
    print(f'投影パターン: {df["projection"].unique()}')
    
    # 図の保存先ディレクトリ
    figure_dir = os.path.join(rootDir, SAVE_DIR, 'figures')
    os.makedirs(figure_dir, exist_ok=True)
    
    # ============================================================================
    # 1. 全データに対する箱ひげ図
    # ============================================================================
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # 投影パターンごとにデータを準備
    projections = ['original', 'modified_1', 'modified_2']
    data_all = [df[df['projection'] == proj]['bpm_MAE'].dropna() for proj in projections]
    
    # 箱ひげ図を作成
    bp1 = ax.boxplot(data_all, tick_labels=projections, patch_artist=True,
                     showmeans=True, meanline=True)
    
    # 色をつける
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    for patch, color in zip(bp1['boxes'], colors):
        patch.set_facecolor(color)
    
    ax.set_xlabel('Projection Matrix', fontsize=12)
    ax.set_ylabel('BPM MAE', fontsize=12)
    ax.set_title(f'Comparison of Projection Matrices - All Data\n({dataName})', fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # 統計情報を追加
    stats_text = []
    for proj, data in zip(projections, data_all):
        stats_text.append(f'{proj}: n={len(data)}, mean={np.mean(data):.2f}, median={np.median(data):.2f}')
    ax.text(0.02, 0.98, '\n'.join(stats_text), transform=ax.transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            fontsize=9)
    
    plt.tight_layout()
    output_path_all = os.path.join(figure_dir, f'projection_comparison_all_{dataName}.png')
    plt.savefig(output_path_all, dpi=300, bbox_inches='tight')
    print(f'\n全データの箱ひげ図を保存: {output_path_all}')
    plt.close()
    
    # ============================================================================
    # 2. Originalが高MAE(MAE>=10)のデータに対する箱ひげ図
    # ============================================================================
    # まず、originalで高MAEのwindow_indexを特定
    original_df = df[df['projection'] == 'original'].copy()
    high_mae_windows = original_df[original_df['bpm_MAE'] >= 10]['window_index'].unique()
    
    print(f'\nOriginalで高MAE(>=10)のウィンドウ数: {len(high_mae_windows)}')
    
    if len(high_mae_windows) > 0:
        # 高MAEウィンドウのデータを抽出
        high_mae_df = df[df['window_index'].isin(high_mae_windows)].copy()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        # 投影パターンごとにデータを準備
        data_high_mae = [high_mae_df[high_mae_df['projection'] == proj]['bpm_MAE'].dropna() 
                         for proj in projections]
        
        # 箱ひげ図を作成
        bp2 = ax.boxplot(data_high_mae, tick_labels=projections, patch_artist=True,
                         showmeans=True, meanline=True)
        
        # 色をつける
        for patch, color in zip(bp2['boxes'], colors):
            patch.set_facecolor(color)
        
        ax.set_xlabel('Projection Matrix', fontsize=12)
        ax.set_ylabel('BPM MAE', fontsize=12)
        ax.set_title(f'Comparison of Projection Matrices - High MAE Windows (Original MAE>=10)\n({dataName})', 
                     fontsize=14)
        ax.grid(True, alpha=0.3)
        ax.axhline(y=10, color='red', linestyle='--', alpha=0.5, label='MAE=10 threshold')
        
        # 統計情報を追加
        stats_text = []
        for proj, data in zip(projections, data_high_mae):
            improvement = ''
            if proj != 'original' and len(data_high_mae[0]) > 0:
                original_mean = np.mean(data_high_mae[0])
                current_mean = np.mean(data)
                improvement = f', Δ={current_mean-original_mean:.2f}'
            stats_text.append(f'{proj}: n={len(data)}, mean={np.mean(data):.2f}, median={np.median(data):.2f}{improvement}')
        
        ax.text(0.02, 0.98, '\n'.join(stats_text), transform=ax.transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                fontsize=9)
        ax.legend()
        
        plt.tight_layout()
        output_path_high = os.path.join(figure_dir, f'projection_comparison_high_mae_{dataName}.png')
        plt.savefig(output_path_high, dpi=300, bbox_inches='tight')
        print(f'高MAEデータの箱ひげ図を保存: {output_path_high}')
        plt.close()
        
        # 改善率の計算と表示
        print(f'\n{"="*60}')
        print(f'改善効果の分析 ({dataName}):')
        print(f'{"="*60}')
        original_mean = np.mean(data_high_mae[0])
        for proj, data in zip(projections[1:], data_high_mae[1:]):
            current_mean = np.mean(data)
            improvement = original_mean - current_mean
            improvement_rate = (improvement / original_mean) * 100 if original_mean > 0 else 0
            print(f'{proj}:')
            print(f'  平均MAE: {current_mean:.2f} (Original: {original_mean:.2f})')
            print(f'  改善量: {improvement:.2f} bpm')
            print(f'  改善率: {improvement_rate:.2f}%')
    else:
        print('\n高MAE(>=10)のウィンドウが存在しないため、2つ目の図はスキップします。')
    
    cap.release()
    print(f'\n{dataName}の処理が完了しました。\n')

全動画統合の箱ひげ図作成


In [ ]:
print('\n' + '='*60)
print('全動画統合の箱ひげ図を作成中...')
print('='*60)

# 全動画のデータを統合
all_movies_df_list = []

for i in range(len(movie_paths)):
    rootDir = data_dirs[i]
    dataName = movie_names[i]
    projection_comparison_path = os.path.join(rootDir, SAVE_DIR, f'projection_comparison_{dataName}.csv')
    
    if os.path.exists(projection_comparison_path):
        temp_df = pd.read_csv(projection_comparison_path)
        temp_df['movie_name'] = dataName  # 動画名を追加
        all_movies_df_list.append(temp_df)
        print(f'{dataName}: {len(temp_df)}件のデータを読み込み')

if len(all_movies_df_list) > 0:
    # 全データを結合
    all_movies_df = pd.concat(all_movies_df_list, ignore_index=True)
    print(f'\n統合データ総数: {len(all_movies_df)}件')
    print(f'動画数: {len(all_movies_df["movie_name"].unique())}')
    print(f'投影パターン: {all_movies_df["projection"].unique()}')
    
    # 保存先ディレクトリ(最初の動画のディレクトリを使用、または共通ディレクトリを指定)
    integrated_figure_dir = os.path.join(SAVE_DIR)
    print(f'\n統合図の保存先ディレクトリ: {integrated_figure_dir}')
    os.makedirs(integrated_figure_dir, exist_ok=True)
    
    projections = ['original', 'modified_1', 'modified_2']
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    
    # ============================================================================
    # 1. 全動画・全データに対する箱ひげ図
    # ============================================================================
    fig, ax = plt.subplots(figsize=(12, 7))
    
    data_integrated_all = [all_movies_df[all_movies_df['projection'] == proj]['bpm_MAE'].dropna() 
                          for proj in projections]
    
    bp1 = ax.boxplot(data_integrated_all, tick_labels=projections, patch_artist=True,
                     showmeans=True, meanline=True)
    
    for patch, color in zip(bp1['boxes'], colors):
        patch.set_facecolor(color)
    
    ax.set_xlabel('Projection Matrix', fontsize=12)
    ax.set_ylabel('BPM MAE', fontsize=12)
    ax.set_title(f'Comparison of Projection Matrices - All Movies Combined\n(Total: {len(all_movies_df["movie_name"].unique())} movies)', 
                 fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # 統計情報を追加
    stats_text = []
    for proj, data in zip(projections, data_integrated_all):
        stats_text.append(f'{proj}: n={len(data)}, mean={np.mean(data):.2f}, median={np.median(data):.2f}, std={np.std(data):.2f}')
    ax.text(0.02, 0.98, '\n'.join(stats_text), transform=ax.transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            fontsize=10)
    
    plt.tight_layout()
    output_path_integrated_all = os.path.join(integrated_figure_dir, 'projection_comparison_all_movies.png')
    plt.savefig(output_path_integrated_all, dpi=300, bbox_inches='tight')
    print(f'\n全動画統合(全データ)の箱ひげ図を保存: {output_path_integrated_all}')
    plt.close()
    
    # ============================================================================
    # 2. 全動画・Originalが高MAE(MAE>=10)のデータに対する箱ひげ図
    # ============================================================================
    # Originalで高MAEのwindow_indexを特定(動画ごとに管理)
    original_integrated_df = all_movies_df[all_movies_df['projection'] == 'original'].copy()
    
    # 動画名とwindow_indexの組み合わせで高MAEを特定
    high_mae_mask = original_integrated_df['bpm_MAE'] >= 10
    high_mae_keys = original_integrated_df[high_mae_mask][['movie_name', 'window_index']].values
    
    print(f'\n全動画でOriginalが高MAE(>=10)のウィンドウ数: {len(high_mae_keys)}')
    
    if len(high_mae_keys) > 0:
        # 高MAEウィンドウのデータを抽出
        high_mae_integrated_df = all_movies_df[
            all_movies_df.apply(lambda row: any((row['movie_name'] == k[0]) and (row['window_index'] == k[1]) 
                                               for k in high_mae_keys), axis=1)
        ].copy()
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        data_integrated_high_mae = [high_mae_integrated_df[high_mae_integrated_df['projection'] == proj]['bpm_MAE'].dropna() 
                                   for proj in projections]
        
        bp2 = ax.boxplot(data_integrated_high_mae, tick_labels=projections, patch_artist=True,
                        showmeans=True, meanline=True)
        
        for patch, color in zip(bp2['boxes'], colors):
            patch.set_facecolor(color)
        
        ax.set_xlabel('Projection Matrix', fontsize=12)
        ax.set_ylabel('BPM MAE', fontsize=12)
        ax.set_title(f'Comparison of Projection Matrices - High MAE Windows (Original MAE>=10)\nAll Movies Combined ({len(high_mae_keys)} windows)', 
                     fontsize=14)
        ax.grid(True, alpha=0.3)
        ax.axhline(y=10, color='red', linestyle='--', alpha=0.5, label='MAE=10 threshold')
        
        # 統計情報と改善効果を追加
        stats_text = []
        original_mean = np.mean(data_integrated_high_mae[0])
        for proj, data in zip(projections, data_integrated_high_mae):
            improvement = ''
            if proj != 'original' and len(data_integrated_high_mae[0]) > 0:
                current_mean = np.mean(data)
                improvement_val = current_mean - original_mean
                improvement_rate = ((original_mean - current_mean) / original_mean) * 100 if original_mean > 0 else 0
                improvement = f', Δ={improvement_val:.2f} ({improvement_rate:+.1f}%)'
            stats_text.append(f'{proj}: n={len(data)}, mean={np.mean(data):.2f}, median={np.median(data):.2f}{improvement}')
        
        ax.text(0.02, 0.98, '\n'.join(stats_text), transform=ax.transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                fontsize=10)
        ax.legend()
        
        plt.tight_layout()
        output_path_integrated_high = os.path.join(integrated_figure_dir, 'projection_comparison_high_mae_all_movies.png')
        plt.savefig(output_path_integrated_high, dpi=300, bbox_inches='tight')
        print(f'全動画統合(高MAE)の箱ひげ図を保存: {output_path_integrated_high}')
        plt.close()
        
        # 改善効果の詳細分析
        print(f'\n{"="*60}')
        print(f'全動画統合の改善効果分析:')
        print(f'{"="*60}')
        for proj, data in zip(projections[1:], data_integrated_high_mae[1:]):
            current_mean = np.mean(data)
            improvement = original_mean - current_mean
            improvement_rate = (improvement / original_mean) * 100 if original_mean > 0 else 0
            
            # 改善されたウィンドウの割合
            original_data = data_integrated_high_mae[0]
            improved_count = sum(data.values < original_data.values)
            improved_rate = (improved_count / len(data)) * 100 if len(data) > 0 else 0
            
            print(f'{proj}:')
            print(f'  平均MAE: {current_mean:.2f} (Original: {original_mean:.2f})')
            print(f'  改善量: {improvement:.2f} bpm')
            print(f'  改善率: {improvement_rate:.2f}%')
            print(f'  改善されたウィンドウ: {improved_count}/{len(data)} ({improved_rate:.1f}%)')
            print()
    else:
        print('\n高MAE(>=10)のウィンドウが存在しないため、2つ目の統合図はスキップします。')
    
    print(f'\n全動画統合の処理が完了しました。')
    print(f'図の保存先: {integrated_figure_dir}')
else:
    print('\n統合するデータが見つかりませんでした。')

print('\n' + '='*60)
print('すべての処理が完了しました。')
print('='*60)

輝度と変えた射影行列のMAEを箱ひげ図にプロット

In [ ]:
# ============================================================================
# 全動画統合の箱ひげ図作成
# ============================================================================
print('\n' + '='*60)
print('全動画統合の箱ひげ図を作成中...')
print('='*60)

# 全動画のデータを統合
all_movies_df_list = []
all_intensity_df_list = []

for i in range(len(movie_paths)):
    rootDir = data_dirs[i]
    dataName = movie_names[i]
    projection_comparison_path = os.path.join(rootDir, SAVE_DIR, f'projection_comparison_{dataName}.csv')
    intensity_comparison_path = os.path.join(rootDir, SAVE_DIR, f'window_intensity_{dataName}.csv')
    
    if os.path.exists(projection_comparison_path):
        temp_df = pd.read_csv(projection_comparison_path)
        temp_df['movie_name'] = dataName  # 動画名を追加
        all_movies_df_list.append(temp_df)
        print(f'{dataName}: {len(temp_df)}件のprojectionデータを読み込み')
    
    if os.path.exists(intensity_comparison_path):
        temp_intensity_df = pd.read_csv(intensity_comparison_path)
        temp_intensity_df['movie_name'] = dataName
        all_intensity_df_list.append(temp_intensity_df)
        print(f'{dataName}: {len(temp_intensity_df)}件のintensityデータを読み込み')

if len(all_movies_df_list) > 0:
    # 全データを結合
    all_movies_df = pd.concat(all_movies_df_list, ignore_index=True)
    print(f'\n統合projectionデータ総数: {len(all_movies_df)}件')
    print(f'動画数: {len(all_movies_df["movie_name"].unique())}')
    print(f'投影パターン: {all_movies_df["projection"].unique()}')
    
    has_intensity_integrated = False
    if len(all_intensity_df_list) > 0:
        all_intensity_df = pd.concat(all_intensity_df_list, ignore_index=True)
        print(f'統合intensityデータ総数: {len(all_intensity_df)}件')
        has_intensity_integrated = True
    
    # 保存先ディレクトリ(最初の動画のディレクトリを使用、または共通ディレクトリを指定)
    integrated_figure_dir = os.path.join(SAVE_DIR)
    os.makedirs(integrated_figure_dir, exist_ok=True)
    
    projections = ['original', 'modified_1', 'modified_2']
    if has_intensity_integrated:
        projections.append('intensity')
    colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
    
    # ============================================================================
    # 1. 全動画・全データに対する箱ひげ図
    # ============================================================================
    fig, ax = plt.subplots(figsize=(14, 7))
    
    data_integrated_all = []
    for proj in projections[:3]:
        data_integrated_all.append(all_movies_df[all_movies_df['projection'] == proj]['bpm_MAE'].dropna())
    if has_intensity_integrated:
        data_integrated_all.append(all_intensity_df['bpm_MAE'].dropna())
    
    # data_integrated_allをのデータ数確認
    print('\n各投影パターンのデータ数:')
    for proj, data in zip(projections, data_integrated_all):
        print(f'  {proj}: {len(data)}件')
    
    df_test = pd.DataFrame({proj: data for proj, data in zip(projections, data_integrated_all)})
    print(df_test.head())
    
    bp1 = ax.boxplot(data_integrated_all, tick_labels=projections, patch_artist=True,
                     showmeans=True, meanline=True)
    
    for patch, color in zip(bp1['boxes'], colors[:len(projections)]):
        patch.set_facecolor(color)
    
    ax.set_xlabel('Projection Matrix / Method', fontsize=12)
    ax.set_ylabel('BPM MAE', fontsize=12)
    ax.set_title(f'Comparison of Projection Matrices and Intensity - All Movies Combined\n(Total: {len(all_movies_df["movie_name"].unique())} movies)', 
                 fontsize=14)
    ax.grid(True, alpha=0.3)
    
    # 統計情報を追加
    stats_text = []
    for proj, data in zip(projections, data_integrated_all):
        stats_text.append(f'{proj}: n={len(data)}, mean={np.mean(data):.2f}, median={np.median(data):.2f}, std={np.std(data):.2f}')
    ax.text(0.02, 0.98, '\n'.join(stats_text), transform=ax.transAxes,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            fontsize=10)
    
    plt.tight_layout()
    output_path_integrated_all = os.path.join(integrated_figure_dir, 'projection_intensity_comparison_all_movies.png')
    plt.savefig(output_path_integrated_all, dpi=300, bbox_inches='tight')
    print(f'\n全動画統合(全データ)の箱ひげ図を保存: {output_path_integrated_all}')
    plt.close()
    
    # ============================================================================
    # 2. 全動画・Originalが高MAE(MAE>=10)のデータに対する箱ひげ図
    # ============================================================================
    # Originalで高MAEのwindow_indexを特定(動画ごとに管理)
    original_integrated_df = all_movies_df[all_movies_df['projection'] == 'original'].copy()
    
    # 動画名とwindow_indexの組み合わせで高MAEを特定
    high_mae_mask = original_integrated_df['bpm_MAE'] >= 10
    high_mae_keys = original_integrated_df[high_mae_mask][['movie_name', 'window_index']].values
    
    if len(high_mae_keys) > 0:
        # 高MAEウィンドウのデータを抽出
        high_mae_integrated_df = all_movies_df[
            all_movies_df.apply(lambda row: any((row['movie_name'] == k[0]) and (row['window_index'] == k[1]) 
                                               for k in high_mae_keys), axis=1)
        ].copy()
        
        fig, ax = plt.subplots(figsize=(14, 7))
        
        data_integrated_high_mae = []
        for proj in projections[:3]:
            data_integrated_high_mae.append(
                high_mae_integrated_df[high_mae_integrated_df['projection'] == proj]['bpm_MAE'].dropna()
            )
        
        if has_intensity_integrated:
            # Intensityデータも同じ(movie_name, window_index)の組み合わせでフィルタ
            intensity_high_mae_integrated = all_intensity_df[
                all_intensity_df.apply(lambda row: any((row['movie_name'] == k[0]) and (row['window_index'] == k[1]) 
                                                       for k in high_mae_keys), axis=1)
            ]['bpm_MAE'].dropna()
            data_integrated_high_mae.append(intensity_high_mae_integrated)
        
        bp2 = ax.boxplot(data_integrated_high_mae, tick_labels=projections, patch_artist=True,
                        showmeans=True, meanline=True)
        
        for patch, color in zip(bp2['boxes'], colors[:len(projections)]):
            patch.set_facecolor(color)
        
        ax.set_xlabel('Projection Matrix / Method', fontsize=12)
        ax.set_ylabel('BPM MAE', fontsize=12)
        ax.set_title(f'Comparison of Projection Matrices and Intensity - High MAE Windows (Original MAE>=10)\nAll Movies Combined ({len(high_mae_keys)} windows)', 
                     fontsize=14)
        ax.grid(True, alpha=0.3)
        ax.axhline(y=10, color='red', linestyle='--', alpha=0.5, label='MAE=10 threshold')
        
        # 統計情報と改善効果を追加
        stats_text = []
        original_mean = np.mean(data_integrated_high_mae[0])
        for proj, data in zip(projections, data_integrated_high_mae):
            improvement = ''
            if proj != 'original' and len(data_integrated_high_mae[0]) > 0:
                current_mean = np.mean(data)
                improvement_val = current_mean - original_mean
                improvement_rate = ((original_mean - current_mean) / original_mean) * 100 if original_mean > 0 else 0
                improvement = f', Δ={improvement_val:.2f} ({improvement_rate:+.1f}%)'
            stats_text.append(f'{proj}: n={len(data)}, mean={np.mean(data):.2f}, median={np.median(data):.2f}{improvement}')
        
        ax.text(0.02, 0.98, '\n'.join(stats_text), transform=ax.transAxes,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
                fontsize=10)
        ax.legend()
        
        plt.tight_layout()
        output_path_integrated_high = os.path.join(integrated_figure_dir, 'projection_intensity_comparison_high_mae_all_movies.png')
        plt.savefig(output_path_integrated_high, dpi=300, bbox_inches='tight')
        print(f'全動画統合(高MAE)の箱ひげ図を保存: {output_path_integrated_high}')
        plt.close()
        
        # 改善効果の詳細分析
        print(f'\n{"="*60}')
        print(f'全動画統合の改善効果分析:')
        print(f'{"="*60}')
        for proj, data in zip(projections[1:], data_integrated_high_mae[1:]):
            current_mean = np.mean(data)
            improvement = original_mean - current_mean
            improvement_rate = (improvement / original_mean) * 100 if original_mean > 0 else 0
            
            # 改善されたウィンドウの割合を計算(同じインデックスで比較可能な場合のみ)
            print(f'{proj}:')
            print(f'  平均MAE: {current_mean:.2f} (Original: {original_mean:.2f})')
            print(f'  改善量: {improvement:.2f} bpm')
            print(f'  改善率: {improvement_rate:.2f}%')
            print()
    else:
        print('\n高MAE(>=10)のウィンドウが存在しないため、2つ目の統合図はスキップします。')
    
    print(f'\n全動画統合の処理が完了しました。')
    print(f'図の保存先: {integrated_figure_dir}')
else:
    print('\n統合するデータが見つかりませんでした。')

print('\n' + '='*60)
print('すべての処理が完了しました。')
print('='*60)

### POS拡張

In [ ]:
methodCombinations = [
    ['cuda', cupy_POS, "cupy_POS"]
]

for i in range(len(movie_paths)):
    print(f'Processing movie: {movie_paths[i]}')
    inputMoviePath = movie_paths[i]
    rootDir = data_dirs[i]
    dataName = movie_names[i]

    cap = cv2.VideoCapture(inputMoviePath)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    window_analysis_path = os.path.join(rootDir, RPPG_ACCURACY_EVAL_DIR, f'window_analysis_{dataName}.csv')
    window_analysis_df = pd.read_csv(window_analysis_path)

    # 手法がcupy_POSのデータを抽出
    high_mae_df = window_analysis_df[window_analysis_df['bvp_method'] == 'cupy_POS'].copy()
    print(f'\n高MAEデータ - {dataName}, 件数: {len(high_mae_df)}')

    # 結果保存用のリスト
    results_list = []
    
    window_index = 0
    # high_mae_dfの各行を処理
    for idx, row in high_mae_df.iterrows():
        window_index = row['window_index']
        window_size = row['window_size']
        stride = row['stride']
        frame_start = row['frame_start']
        frame_end = row['frame_end']
        window_start_time = row['window_start_time']
        window_end_time = row['window_end_time']
        ecg_bpm_in_window_mean = row['ecg_bpm_mean']
        bpm_MAE = row['bpm_MAE']

        print(f"\nウィンドウ {window_index}: 窓サイズ {window_size}s, ストライド {stride}s, フレーム {frame_start}-{frame_end}, 時間 {window_start_time:.2f}-{window_end_time:.2f}s")

        # RGB信号を抽出
        r_signal_in_window = np.fromstring(row['r_signal_in_window'][1:-1], sep=' ')
        g_signal_in_window = np.fromstring(row['g_signal_in_window'][1:-1], sep=' ')
        b_signal_in_window = np.fromstring(row['b_signal_in_window'][1:-1], sep=' ')

        print(f'  R信号長: {len(r_signal_in_window)}, G信号長: {len(g_signal_in_window)}, B信号長: {len(b_signal_in_window)}')

        # rgbからBVPを計算
        rgb_signal = np.array([[r_signal_in_window, g_signal_in_window, b_signal_in_window]], dtype=np.float32)
        print(f"\nRGB信号の形状: {rgb_signal.shape}")

        signal_length = rgb_signal.shape[2]
        min_required_length = 50
            
        filtered_signal = [rgb_signal]
        
        # ============================================================================
        # 【重要】POS法を直接実装(調整可能)
        # ============================================================================
        import cupy as cp

        # CuPy配列に変換
        rgb_cupy = cp.asarray(rgb_signal)

        # POS法のパラメータ
        eps = 10**-9
        X = rgb_cupy
        fps_cupy = cp.float32(fps)
        e, c, f = X.shape  # e = #estimators, c = 3 rgb ch., f = #frames
        w = int(1.6 * fps_cupy)  # window length

        # 投影行列P(現在のパターンを使用)
        P = cp.array([[0, 1, -1], [-2, 1, 1]], dtype=cp.float32)  # ← 修正: cpを使用
        Q = cp.stack([P for _ in range(e)], axis=0)  # ← 修正: P_cupyではなくP

        # 初期化
        H = cp.zeros((e, f))

        # 診断情報を保存するリスト（各ウィンドウごとに初期化）← 重要！
        M_list = []
        window_indices = []

        # スライディングウィンドウループ
        for n in cp.arange(w, f):
            m = n - w + 1
            
            # 時間的正規化
            Cn = X[:, :, m:(n + 1)]
            # POSの原理式のNに相当(N・us・I0のN)
            M = 1.0 / (cp.mean(Cn, axis=2) + eps)
            M_expanded = cp.expand_dims(M, axis=2)
            Cn = cp.multiply(M_expanded, Cn)
            
            # Mの値を保存（形状: (e, c) = (1, 3)）
            M_list.append(cp.asnumpy(M))
            
            # 投影
            S = cp.dot(Q, Cn)
            S = S[0, :, :, :]
            S = cp.swapaxes(S, 0, 1)
            
            # チューニング
            S1 = S[:, 0, :]
            S2 = S[:, 1, :]
            alpha = cp.std(S1, axis=1) / (eps + cp.std(S2, axis=1))
            
            alpha_expanded = cp.expand_dims(alpha, axis=1)
            alpha_S2 = alpha_expanded * S2
            
            window_indices.append(int(n))
            
            Hn = cp.add(S1, alpha_S2)
            Hnm = Hn - cp.expand_dims(cp.mean(Hn, axis=1), axis=1)
            
            # オーバーラップ加算
            H[:, m:(n + 1)] = cp.add(H[:, m:(n + 1)], Hnm)

        # NumPy配列に戻す
        bvp_cupy = H
        bvp_numpy = cp.asnumpy(bvp_cupy)

        raw_bvp_signal = [bvp_numpy]
        bvp_signal = [bvp_numpy.copy()]

        # 後処理フィルタリング
        bvp_signal = vhr.BVP.apply_filter(
            bvp_signal,
            vhr.BVP.BPfilter,
            params={'order': 6, 'minHz': 0.5, 'maxHz': 2.0, 'fps': fps}
        )

        bvp_signal = vhr.BVP.apply_filter(bvp_signal, vhr.BVP.zeromean)

        raw_bvp_signal_in_window = raw_bvp_signal[0] if len(bvp_signal) > 0 else None
        filtered_bvp_signal_in_window = bvp_signal[0] if len(bvp_signal) > 0 else None

        # FFT解析
        raw_bvp_signal_in_window = raw_bvp_signal_in_window.flatten() if raw_bvp_signal_in_window is not None else None
        filtered_bvp_signal_in_window = filtered_bvp_signal_in_window.flatten()
        fft_result_dic = analyze_window_fft(filtered_bvp_signal_in_window, fps)

        # MAEの計算
        rppg_bpm = fft_result_dic['max_freq'] * 60
        rppg_freq = fft_result_dic['frequencies']
        rppg_amplitude = fft_result_dic['amplitudes']
        rppg_pwd = fft_result_dic['power_spectrum']

        bpm_MAE = np.abs(ecg_bpm_in_window_mean - rppg_bpm) if not np.isnan(ecg_bpm_in_window_mean) else np.nan

        proj_name = "standard_POS"  # ← 追加
        
        print(f"\n結果サマリー [{proj_name}]:")
        print(f"  ECG BPM: {ecg_bpm_in_window_mean:.2f}")
        print(f"  rPPG BPM: {rppg_bpm:.2f}")
        print(f"  MAE: {bpm_MAE:.2f}")
        print(f"  M_listの長さ: {len(M_list)}")  # ← デバッグ用
        
        # M_listを文字列に変換（CSV保存用）
        # 各Mは形状(1,3)なので、フラット化してスペース区切りに
        M_list_str = [' '.join(map(str, M.flatten())) for M in M_list]
        M_list_combined = '; '.join(M_list_str)  # セミコロンで各時刻を区切る
        
        # 結果を辞書形式でリストに追加
        result_dict = {
            'window_index': window_index,
            'projection': proj_name,
            'window_size': window_size,
            'stride': stride,
            'frame_start': frame_start,
            'frame_end': frame_end,
            'window_start_time': window_start_time,
            'window_end_time': window_end_time,
            'ecg_bpm': ecg_bpm_in_window_mean,
            'rppg_bpm': rppg_bpm,
            'bpm_MAE': bpm_MAE,
            'M_list': M_list_combined,  # ← 文字列化して保存
            'num_sliding_windows': len(M_list),  # ← スライディングウィンドウ数
        }
        results_list.append(result_dict)
    
    # 全ウィンドウの処理が終わった後、DataFrameに変換
    results_df = pd.DataFrame(results_list)
    
    # 結果をCSVに保存
    output_path = os.path.join(rootDir, SAVE_DIR, f'POS_internalArray_output_{dataName}.csv')
    results_df.to_csv(output_path, index=False)
    print(f"\n結果を保存しました: {output_path}")

In [ ]:
# 窓ごとにR,G,BのM_listの変化をプロット
